In [1]:
import pandas as pd
import re

***
# 1. Normalizacion de la tabla Municipios 

 Descripción de la data. Tabla Municipios.xlsx
1.	**DP:** Código numérico que identifica al Departamento (en este caso, 5 corresponde a Antioquia).
2.	**Departamento:** Nombre de la división político-administrativa principal (equivalente a estados o provincias en otros países). En los datos mostrados aparece "Antioquia" con algunas variaciones tipográficas.
3.	**MPIO:** Código numérico de 4 dígitos que identifica al Municipio dentro del departamento (por ejemplo, 5001, 5002, etc.).
4.	**Municipio:** Nombre de la división político-administrativa menor dentro del departamento (por ejemplo, Medellín, Abejorral, etc.).
5.	**Superficie:** Extensión territorial del municipio medida en kilómetros cuadrados (km²).
6.	**PopTot:** Población total del municipio (número de habitantes).
7.	**Rural:** Porcentaje de la población que vive en zonas rurales del municipio.
8.	**Región:** Agrupación geográfica más amplia a la que pertenece el municipio y departamento



In [2]:
##############################################
###   Funcion de normalizacion de textos   ###
##############################################
# Objetivo de la funcion es normalizar los campos: Departamento y Municipio

def normalizar_texto(texto):
    # 1. Eliminar caracteres especiales pero conservar letras con acentos, ñ y espacios
    texto_limpio = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚüÜñÑ\s]', '', texto)
    # 2. Normalizar espacios múltiples a un solo espacio
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio)
    # 3. Aplicar formato de título (primera letra de cada palabra en mayúscula)
    texto_normalizado = texto_limpio.title()
    # 4. Eliminar espacios al inicio y final
    texto_normalizado = texto_normalizado.strip()
    return texto_normalizado

# Lista de ejemplos
ejemplos = ["Ant%ioq>uia", "Ant%ioquia", "Ant%ioqUia", "Antioq>uia", 
            "Antioquia", "Ant%io>qUia", "AntioqUia", "Ant%io>quia",
            "Vaup>és", "Bogot%á", "Córd>oba", "Nariñ%o",
            "Norte   de   San>tander", "Valle  del   Cau>ca"]

# Aplicar normalización a cada ejemplo
for texto in ejemplos:
    print(f"{texto:20} -> {normalizar_texto(texto)}")

Ant%ioq>uia          -> Antioquia
Ant%ioquia           -> Antioquia
Ant%ioqUia           -> Antioquia
Antioq>uia           -> Antioquia
Antioquia            -> Antioquia
Ant%io>qUia          -> Antioquia
AntioqUia            -> Antioquia
Ant%io>quia          -> Antioquia
Vaup>és              -> Vaupés
Bogot%á              -> Bogotá
Córd>oba             -> Córdoba
Nariñ%o              -> Nariño
Norte   de   San>tander -> Norte De Santander
Valle  del   Cau>ca  -> Valle Del Cauca


***
# 2. La tabla municipios tiene dos campos a normalizar: 
* Departamento
* Municipio

In [3]:
Municipios = pd.read_excel( "01_PRUEBA_IETS\\02_Municipios.xlsx")   
Municipios['Departamento'] = Municipios['Departamento'].apply(normalizar_texto)
Municipios['Municipio'] = Municipios['Municipio'].apply(normalizar_texto).copy()
Municipios.shape

(1118, 8)

***
###  Validacion, integridad referencial para los campos (Departamento, Municipio)

In [4]:
Municipios.head(4)

,DP,Departamento,MPIO,Municipio,Superficie,PopTot,Rural,Region
0,5,Antioquia,5001,Medellín,350.666623,2634570,1.6,Región Eje Cafetero
1,5,Antioquia,5002,Abejorral,497.566212,21622,56.7,Región Eje Cafetero
2,5,Antioquia,5004,Abriaquí,287.641603,2872,64.2,Región Eje Cafetero
3,5,Antioquia,5021,Alejandría,119.467683,4989,40.7,Región Eje Cafetero


In [5]:
# Se comprueba que colo hay una referencia del campo "Departamento" por cada codigo "DP"
# Si el numero es 1 se ccomprueba que hay una relacion 1 a 1 para los codigos "DP" para Cada descripcion de Departamento  
Municipios.groupby("DP").nunique()['Departamento'].max()

np.int64(1)

In [6]:
# Se comprueba que colo hay una referencia del campo "Municipio" por cada codigo "MPIO"
Municipios.groupby("MPIO").nunique()["Municipio"].max()

np.int64(1)

***
# 3. Normalizacion de la informacion numerica para los campos:
  
- Superficie
- PopTot
- Rural

Se determina que la forma más adecuada de imputar los datos faltantes es mediante un modelo de lenguaje grande (LLM), considerando la naturaleza georreferenciada de la información.
Para este ejemplo se utiliza el ultimo de de openai que es el *gpt-4.5-preview*


In [ ]:
import openai

# Asigna la clave de API
apikey =  "Para verificar este codigo, solicitenme la apy KEY, ya que al publicarlo en GITHUB la dejaria expuesta, y tengo creditos"

openai.api_key = apikey  # Reemplaza con tu clave real
def GPTPROMPT_PLUS(GPTprompt, contenido):
    response = openai.ChatCompletion.create(model = "gpt-4.5-preview", #"gpt-4.1", # "gpt-4o-search-preview" ,# REVISAR LOS COSTOS, mequedan poquitos creditos en la API 
        #web_search_options={"search_context_size": "high"},  # low medium  high
        messages=[{"role": "system", "content": f"""
                   Proporciona datos georeferenciados y demográficos actualizados de Colombia: 
                 
                   :\n[{contenido}]"""},
                  {"role": "user", "content": GPTprompt}])
    return response["choices"][0]["message"]["content"]  # Extrae la respuesta correctamente

In [8]:
GPTprompt = """
                Genera un archivo JSON con la siguiente información basada en la descripción del medicamento proporcionada.

                    No incluyas ningún texto adicional antes ni después del JSON.
                    Si algún campo no está disponible o no aplica, usa el valor "NO_APLICA".
                    En caso de que falte información, realiza una búsqueda en la web para completar los datos.
                    
                    Los campos requeridos en el JSON son:

                            "Superficie": "Extensión territorial del municipio medida en kilómetros cuadrados (km²), no incluir unidades, solo un numero tipo float",
                            "PopTot": "Población total del municipio (número de habitantes), solo un numero tipo float",
                            "Rural": "Porcentaje de la población que vive en zonas rurales del municipio. solo un numero tipo float, porcentale es un numero de 0 a 100"
                    """

In [9]:
import json
import re

def limpiar_respuesta_api(respuesta):
    """
    Limpia la respuesta de OpenAI eliminando delimitadores de código 
    y asegurando que sea un JSON válido.
    """
    # 1. Eliminar delimitadores de código ```json y ```
    respuesta = re.sub(r"```json|```", "", respuesta, flags=re.IGNORECASE).strip()
    
    # 2. Verificar si el JSON es válido antes de convertirlo
    try:
        json_data = json.loads(respuesta)
        return json_data  # Retornar JSON válido como diccionario
    except json.JSONDecodeError:
        return None  # Si falla, retorna None para manejo de errores


In [10]:
################################################################
#######        Se identifica informacion faltante        #######
################################################################
# Se identifica informacion Faltante teniendo en cuenta valores nulos y se rectifican los valores en cero.  
#  
VALIDACION = Municipios[(Municipios.isna() | (Municipios == 0)).any(axis=1)]
VALIDACION

,DP,Departamento,MPIO,Municipio,Superficie,PopTot,Rural,Region
125,8,Atlántico,8001,Barranquilla,161.872561,1342818,0.0,Región Caribe
1097,94,Guainía,94663,Mapiripana,NaN,0,NaN,Región Llano


***
# Imputacion automatizada de informacion faltante
- La siguiente funcion busca hacer la imputacion condicionada de campos 
- Toma, solamente los campos NULL o los campos con 0 y los reemplza por la informacion consultada mediante los modelos LLM de openai


In [11]:

for indice in VALIDACION.index:
    print("-" * 24 , "\n", "index: ", indice)
    contenido = VALIDACION.loc[indice][['DP', 'Departamento', 'MPIO', 'Municipio']].to_string()
    print (contenido)
    DICCIONARIO = limpiar_respuesta_api(GPTPROMPT_PLUS(GPTprompt, contenido))
    print("\nResultado consulta: ", DICCIONARIO)
    for campo in DICCIONARIO.keys():
        if pd.isna(Municipios.at[indice, campo]) or Municipios.at[indice, campo] == 0:  #  or Municipios.at[indice, campo] == 0
            Municipios.at[Municipios.index[indice], campo] = DICCIONARIO[campo]

------------------------ 
 index:  125
DP                         8
Departamento       Atlántico
MPIO                    8001
Municipio       Barranquilla

Resultado consulta:  {'Superficie': 154.0, 'PopTot': 1326582.0, 'Rural': 0.0}
------------------------ 
 index:  1097
DP                      94
Departamento       Guainía
MPIO                 94663
Municipio       Mapiripana

Resultado consulta:  {'Superficie': 11780.0, 'PopTot': 3533.0, 'Rural': 93.8}


In [12]:
Municipios.to_excel("02_Tablas_Normalizadas\\02_Municipios Normalizado.xlsx") 